In [35]:
import json
import pandas as pd
import numpy as np
import os
from os import path
import time
import requests
import re
from collections import defaultdict
import gspread

In [2]:
def get_size(size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony/validators_with_page?active=false&page=0&search=&size={}&sortOrder=desc&sortProperty=apr".format(size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['total']

In [3]:
def get_validator(page, size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony/validators_with_page?active=false&page={}&search=&size={}&sortOrder=desc&sortProperty=apr".format(page, size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['validators']

In [4]:
# get uptime
size = get_size(1)
pages = size // 100 + 1
# get the validator info
validator = []
for i in range(pages):
    val = get_validator(i,100)
    validator.extend(val)

In [40]:
column = list(validator[0].keys())
remove = ['bls-public-keys','delegations','last-epoch-in-committee','update-height','remainder','voting_power','signed_blocks','blocks_should_sign','uctDate','index','commision-recent-change','average_stake','average_stake_by_bls','active', 'epoch_apr']
new_column = []
for i in column:
    if i not in remove:
        new_column.append(i)
df = pd.DataFrame(validator, columns = new_column)
percent_change = ['rate','max-rate','max-change-rate', 'apr','last_apr','uptime_percentage']
for name in percent_change:
    df[name] = df[name].apply(lambda c: None if np.isnan(float(c)) else "{0:.2f}%".format(float(c)*100))
atto_change = ['min-self-delegation','max-total-delegation', 'self_stake', 'total_stake', 'lifetime_reward_accumulated']
for name in atto_change:
    df[name] = df[name].apply(lambda c: float(c)/1e18)
# df.to_csv('../csv/validator_info_mainnet.csv', index = False)
# print(f"csv file saved in http://analytics.hmny.io/edit/harmony-log-analysis/projects/staking_dashboard/csv/validator_info_mainnet.csv")

In [41]:
df

,min-self-delegation,max-total-delegation,rate,max-rate,max-change-rate,name,identity,website,security-contact,details,creation-height,address,self_stake,total_stake,active_nodes,elected_nodes,uptime_percentage,last_apr,lifetime_reward_accumulated,apr
0,10000.0,1.000000e+09,2.00%,100.00%,5.00%,Mind Heart Soul,MindHeartSoul,https://ict.mind-heart-soul.org,Koen,"Well-being & tech work, actively seeking to ma...",3374707,one1efld6lrq5huppf35skdhgc9ek0kfv99klqf7vv,50005.000,3.100169e+07,4,4,98.09%,11.14%,114415.354165,10.32%
1,10000.0,1.000000e+08,9.50%,50.00%,5.00%,Tempo,TempoValidator,,,We run dual redundant harmony nodes on enterpr...,3935644,one1xx7d6ldrtla7dw7yawg4rq05ws05qrkhxxfqaa,370000.000,3.700000e+05,1,0,None,0.00%,0.000000,None
2,10000.0,1.000000e+08,5.00%,15.00%,2.00%,CryptoMason,Mason,https://t.me/harmonymason,@valentincrypt,https://t.me/harmonymason,3362251,one18je40sxd9s6awtgx4akhuda8d75km3663eep37,0.000,0.000000e+00,1,0,98.73%,137.67%,15095.871547,114.01%
3,10000.0,1.000000e+08,0.88%,50.00%,10.00%,Asymmetry Ventures,Asymmetry Ventures,asymmetry.ventures,harmony@asymmetry.ventures,Staking Made Easy. 100% Commitment to Uptime a...,3372909,one180mfv4dneefp9g74duxhspjvkmcjffstd0sj6q,0.000,0.000000e+00,1,0,98.96%,134.61%,15556.096543,111.86%
4,10000.0,1.000000e+09,5.00%,20.00%,10.00%,Coff33Blak,Coff33Blak,channing@harmony.one,Channing,Channing,3361912,one1h2vnc52gdwqa5w6z4gwtrdgsgq8f5fj7aev3f2,0.000,0.000000e+00,1,0,98.51%,67.19%,23679.533326,104.46%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,10000.0,1.000000e+08,100.00%,100.00%,10.00%,zegen.org,zegen.org by ankr,www.ankr.com,info@ankr.com,This validator is launched from app.ankr.com,3933338,one1pcqpsdrlu0p84xx32nmwzqmfz266k5e4knka7a,1000000.000,1.000000e+06,1,0,None,0.00%,0.000000,None
386,10000.0,1.000000e+08,1.00%,10.00%,10.00%,Validator For People,Validator For People,,,Affordable validator for people,3360621,one1ds5h88zazdwnc8tk8j2m0t0wk90x23jcj2dyks,0.000,0.000000e+00,1,0,None,0.00%,0.000000,None
387,10000.0,2.000000e+08,2.50%,50.00%,10.00%,ENTER Group,Harmony IndONEsia,tinyurl.com/yc3u4c9q,winahyos@gmail.com,"Hi All, lets delegate to my validator. Validat...",3418048,one14gevvn4z6vmz6htkxwp62pe77c3dwj3p2cvrv6,18170.000,1.817000e+04,1,0,None,0.00%,0.000000,None
388,10000.0,1.000000e+08,1.00%,10.00%,10.00%,td17,td17 by ankr,www.ankr.com,info@ankr.com,This validator is launched from app.ankr.com,3395576,one1jg5rezn9tqdgpg0awewu9khrg2y9alu3jjpc4e,11013.978,1.101398e+04,1,0,None,0.00%,0.000000,None


In [42]:
gc = gspread.service_account('/home/ubuntu/jupyter/harmony-log-analysis/projects/staking_dashboard/credential/jsonFileFromGoogle.json')
sh = gc.open("harmony-mainnet-tracker")
worksheet = sh.worksheet("validator-dashboard-tracker")
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1AyYHWSkKOCzMY0ZvoT049DapIDvkEhpnfbA1WidJm3o',
 'updatedRange': "'validator-dashboard-tracker'!A1:T391",
 'updatedRows': 391,
 'updatedColumns': 20,
 'updatedCells': 7296}

In [8]:
df

,min-self-delegation,max-total-delegation,rate,max-rate,max-change-rate,name,identity,website,security-contact,details,creation-height,address,self_stake,total_stake,active_nodes,elected_nodes,uptime_percentage,last_apr,lifetime_reward_accumulated,apr
0,10000.0,1.000000e+09,2.00%,100.00%,5.00%,Mind Heart Soul,MindHeartSoul,https://ict.mind-heart-soul.org,Koen,"Well-being & tech work, actively seeking to ma...",3374707,one1efld6lrq5huppf35skdhgc9ek0kfv99klqf7vv,50005.000,3.100169e+07,4,4,98.09%,0.111382399063438238,114415.354165,10.32%
1,10000.0,1.000000e+08,9.50%,50.00%,5.00%,Tempo,TempoValidator,,,We run dual redundant harmony nodes on enterpr...,3935644,one1xx7d6ldrtla7dw7yawg4rq05ws05qrkhxxfqaa,370000.000,3.700000e+05,1,0,nan%,0.000000000000000000,0.000000,nan%
2,10000.0,1.000000e+08,5.00%,15.00%,2.00%,CryptoMason,Mason,https://t.me/harmonymason,@valentincrypt,https://t.me/harmonymason,3362251,one18je40sxd9s6awtgx4akhuda8d75km3663eep37,0.000,0.000000e+00,1,0,98.73%,1.376657007627364201,15095.871547,114.01%
3,10000.0,1.000000e+08,0.88%,50.00%,10.00%,Asymmetry Ventures,Asymmetry Ventures,asymmetry.ventures,harmony@asymmetry.ventures,Staking Made Easy. 100% Commitment to Uptime a...,3372909,one180mfv4dneefp9g74duxhspjvkmcjffstd0sj6q,0.000,0.000000e+00,1,0,98.96%,1.346072673285968580,15556.096543,111.86%
4,10000.0,1.000000e+09,5.00%,20.00%,10.00%,Coff33Blak,Coff33Blak,channing@harmony.one,Channing,Channing,3361912,one1h2vnc52gdwqa5w6z4gwtrdgsgq8f5fj7aev3f2,0.000,0.000000e+00,1,0,98.51%,0.671925942245144461,23679.533326,104.46%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,10000.0,1.000000e+08,100.00%,100.00%,10.00%,zegen.org,zegen.org by ankr,www.ankr.com,info@ankr.com,This validator is launched from app.ankr.com,3933338,one1pcqpsdrlu0p84xx32nmwzqmfz266k5e4knka7a,1000000.000,1.000000e+06,1,0,nan%,0.000000000000000000,0.000000,nan%
386,10000.0,1.000000e+08,1.00%,10.00%,10.00%,Validator For People,Validator For People,,,Affordable validator for people,3360621,one1ds5h88zazdwnc8tk8j2m0t0wk90x23jcj2dyks,0.000,0.000000e+00,1,0,nan%,0.000000000000000000,0.000000,nan%
387,10000.0,2.000000e+08,2.50%,50.00%,10.00%,ENTER Group,Harmony IndONEsia,tinyurl.com/yc3u4c9q,winahyos@gmail.com,"Hi All, lets delegate to my validator. Validat...",3418048,one14gevvn4z6vmz6htkxwp62pe77c3dwj3p2cvrv6,18170.000,1.817000e+04,1,0,nan%,0.000000000000000000,0.000000,nan%
388,10000.0,1.000000e+08,1.00%,10.00%,10.00%,td17,td17 by ankr,www.ankr.com,info@ankr.com,This validator is launched from app.ankr.com,3395576,one1jg5rezn9tqdgpg0awewu9khrg2y9alu3jjpc4e,11013.978,1.101398e+04,1,0,nan%,0.000000000000000000,0.000000,nan%


## email address

In [6]:
pattern = re.compile('(.+?@.+?)')
address = []
for i in df['security-contact']:
    if re.match(pattern, i):
        if "&" in i:
            lst = i.split("&")
            for l in lst:
                address.append(l.replace(" ",""))
        else:
            address.append(i)
with open("../csv/address.txt", "w") as f:
    for i in set(address):
        f.write(i)
        f.write("\n")